<a href="https://colab.research.google.com/github/Rakhshindaa/Internship_tasks/blob/main/Big_data_analysis_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql.functions import *

In [ ]:
# import necessary libraries and Create Spark Session
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName('data_preprocessing').getOrCreate()

In [ ]:
data= sparkSession.read.csv("students performance.csv", header=True)

In [ ]:
data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: string (nullable = true)
 |-- reading score: string (nullable = true)
 |-- writing score: string (nullable = true)



In [ ]:
data.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|            master's degree|    standard|                   none|        66|           62|           63|
|  male|       group D|           some high school|    standard|                   none|        95|           84|           86|
|  male|       group D|          bachelor's degree|    standard|              completed|      NULL|           70|           66|
|  male|       group D|               some college|    standard|                   none|        66|           66|           62|
|female|       group B|               some college|    standard|              completed|        60|     

In [ ]:
data.dtypes

[('gender', 'string'),
 ('race/ethnicity', 'string'),
 ('parental level of education', 'string'),
 ('lunch', 'string'),
 ('test preparation course', 'string'),
 ('math score', 'string'),
 ('reading score', 'string'),
 ('writing score', 'string')]

In [ ]:
data.describe().show()

+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|   100|           100|                        100|         100|                    100|                89|                93|               92|
|   mean|  NULL|          NULL|                       NULL|        NULL|                   NULL| 67.33707865168539| 69.12903225806451|67.44565217391305|
| stddev|  NULL|          NULL|                       NULL|        NULL|                   NULL|15.982905513124667|15.175943353280926|15.45384061260777|
|    min|female|       group A|         associate's degree|free/reduced|          

In [ ]:
# Dropping Entire rows containing Null
null_dropped=data.na.drop()
null_dropped.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|            master's degree|    standard|                   none|        66|           62|           63|
|  male|       group D|           some high school|    standard|                   none|        95|           84|           86|
|  male|       group D|               some college|    standard|                   none|        66|           66|           62|
|female|       group B|               some college|    standard|              completed|        60|           76|           73|
|  male|       group C|               some college|free/reduced|                   none|        41|     

In [ ]:
# Fill Null values inside 'math score'and 'writing score' column with the word 'absent'
data.na.fill('absent',subset=['math score','writing score']).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|            master's degree|    standard|                   none|        66|           62|           63|
|  male|       group D|           some high school|    standard|                   none|        95|           84|           86|
|  male|       group D|          bachelor's degree|    standard|              completed|    absent|           70|           66|
|  male|       group D|               some college|    standard|                   none|        66|           66|           62|
|female|       group B|               some college|    standard|              completed|        60|     

In [ ]:
# Calculate the average of maths score group by gender
avg_maths = data.groupBy("gender").agg(avg("math score"))
avg_maths.show()

+------+-----------------+
|gender|  avg(math score)|
+------+-----------------+
|female|64.53846153846153|
|  male|            69.52|
+------+-----------------+



In [ ]:
# count of students group by lunch
lunch_count= data.groupBy("lunch").agg(count('*'))
lunch_count.show()

+------------+--------+
|       lunch|count(1)|
+------------+--------+
|free/reduced|      28|
|    standard|      72|
+------------+--------+



In [ ]:
# Count of students group by parental level of education
avg_lunch = data.groupBy("parental level of education").agg(count('*'))
avg_lunch.show()

+---------------------------+--------+
|parental level of education|count(1)|
+---------------------------+--------+
|           some high school|      19|
|         associate's degree|      22|
|                high school|      16|
|          bachelor's degree|      18|
|            master's degree|       9|
|               some college|      16|
+---------------------------+--------+



In [ ]:
from pyspark.sql.types import IntegerType
# Cast 'math score' column to IntegerType
data = data.withColumn(" score", data["math score"].cast(IntegerType()))
#Finding the details of students with poor performance (less than 35 marks) in math
poor_performance=data.filter(data["math score"]<35)
poor_performance.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score| score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|female|       group C|               some college|free/reduced|                   none|        32|           46|           48|    32|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+



In [ ]:
#Finding the details of students with highest performance (more than 90 marks) in math
best_performance=data.filter(data["math score"]>=90)
best_performance.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score| score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|  male|       group D|           some high school|    standard|                   none|        95|           84|           86|    95|
|  male|       group D|            master's degree|    standard|                   none|        96|           92|         NULL|    96|
|  male|       group D|         associate's degree|    standard|              completed|       100|           97|           99|   100|
|  male|       group E|          bachelor's degree|    standard|              completed|        99|          100|         NULL|    99|
|female|       group D|         associate's degree|    

In [ ]:
from pyspark.sql import functions as f
print("The number of missing values in each column :")
data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in data.columns]).show()

The number of missing values in each column :
+------+--------------+---------------------------+-----+-----------------------+----------+-------------+-------------+------+
|gender|race/ethnicity|parental level of education|lunch|test preparation course|math score|reading score|writing score| score|
+------+--------------+---------------------------+-----+-----------------------+----------+-------------+-------------+------+
|     0|             0|                          0|    0|                      0|        11|            7|            8|    11|
+------+--------------+---------------------------+-----+-----------------------+----------+-------------+-------------+------+



In [ ]:
data.groupBy("race/ethnicity", "parental level of education").count().show()

+--------------+---------------------------+-----+
|race/ethnicity|parental level of education|count|
+--------------+---------------------------+-----+
|       group A|            master's degree|    1|
|       group D|            master's degree|    2|
|       group D|           some high school|    6|
|       group E|          bachelor's degree|    4|
|       group E|                high school|    3|
|       group A|         associate's degree|    1|
|       group E|         associate's degree|    4|
|       group A|                high school|    1|
|       group B|         associate's degree|    4|
|       group E|            master's degree|    1|
|       group E|           some high school|    2|
|       group D|                high school|    5|
|       group A|           some high school|    2|
|       group C|          bachelor's degree|    4|
|       group D|          bachelor's degree|    5|
|       group B|            master's degree|    1|
|       group B|               

In [ ]:
data.filter(data["test preparation course"] == "completed").show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score| score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------+
|  male|       group D|          bachelor's degree|    standard|              completed|      NULL|           70|           66|  NULL|
|female|       group B|               some college|    standard|              completed|        60|           76|           73|    60|
|  male|       group D|           some high school|    standard|              completed|      NULL|           62|           62|  NULL|
|female|       group C|            master's degree|    standard|              completed|        87|           91|           99|    87|
|female|       group D|         associate's degree|free

In [ ]:
data.select(mean("math score"),max("math score"),min("math score")).show()

+-----------------+---------------+---------------+
|  avg(math score)|max(math score)|min(math score)|
+-----------------+---------------+---------------+
|67.33707865168539|            100|             32|
+-----------------+---------------+---------------+

